In [ ]:
"""
>>> asyncio.run(obj.submit_order(symbol='ADAUSDT', side='SELL', price=1.015, quantity=5, order_type='STOP_MARKET'))
{'orderId': 46915674924, 'symbol': 'ADAUSDT', 'status': 'NEW', 'clientOrderId': '8GSV2rVnGLMBnO4HDwSb1Y', 'price': '0.00000', 'avgPrice': '0.00', 'origQty': '5', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'STOP_MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '1.01500', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'STOP_MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1733915429075}
>>> asyncio.run(obj.submit_order(symbol='ADAUSDT', side='SELL', quantity=5, order_type='MARKET'))
{'orderId': 46916207450, 'symbol': 'ADAUSDT', 'status': 'NEW', 'clientOrderId': '476bSE3TUnGfOp6AsVCq12', 'price': '0.00000', 'avgPrice': '0.00', 'origQty': '5', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0.00000', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0.00000', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'EXPIRE_MAKER', 'goodTillDate': 0, 'updateTime': 1733916764850}
>>>
"""
from dataclasses import dataclass, fields
from typing import List, Optional, Union
import numpy as np
from pprint import pprint
import copy

@dataclass
class TradeOrder:
    symbol: str
    start_timestamp: int  #0
    entry_price: Union[float, int]  #1
    position: int   #2
    quantity: Union[float, int] #3
    leverage: int   #4
    fee_rate: float = 0.05  #5
    end_timestamp:Optional[int] = None    #6
    init_value: Optional[float] = None  #7
    current_value: Optional[float] = None   #8
    profit_and_loss: Optional[float] = None #9
    current_price: Optional[Union[float, int]] = None   #10
    break_event_price: Optional[float] = None   #11
    fee_open: Optional[Union[float, int]] = None    #12
    fee_close: Optional[Union[float, int]] = None   #13

    # 기초정보 오류 검토 및 fee, value업데이트 반영
    def __post_init__(self):
        if self.current_price is None:
            self.current_price = self.entry_price
        if self.leverage <= 0:
            raise ValueError(f"레버리지는 최소 1 이상이어야 합니다. 현재 값: {self.leverage}")
        if self.end_timestamp is None:
            self.end_timestamp = self.start_timestamp
        self.__update_fee()
        self.__update_value()

    # fee 값 정보를 업데이트한다.
    def __update_fee(self):
        # 수수료율 환산
        adjusted_fee_rate = self.fee_rate / 100
        # 매수 수수료 업데이트
        self.fee_open = self.entry_price * adjusted_fee_rate * self.quantity * self.leverage
        # 매도 수수료 업데이트
        self.fee_close = self.current_price * adjusted_fee_rate * self.quantity * self.leverage
        # 수수료비용 합계
        total_fees = self.fee_open + self.fee_close
        #수수료 합계 비용 반영
        self.break_event_price = self.entry_price + (total_fees / self.quantity)

    # value 값 정보를 업데이트한다.
    def __update_value(self):
        # 매수 비용 (수수료 제외)
        self.init_value = (self.entry_price * self.quantity) / self.leverage
        # 평가 가격 (수수료 제외)
        self.current_value = (self.current_price * self.quantity) / self.leverage
        # 수수료비용 합계
        total_fees = self.fee_open + self.fee_close
        # 수수료 합계 비용 선반영
        self.profit_and_loss = self.current_value - self.init_value - total_fees

    # 현재 가격 및 현재 timestamp를 업데이트한다.
    def update_data(self, current_price: [Union[float, int]], current_timestamp: int):
        self.current_price = current_price
        self.end_timestamp = current_timestamp
        self.__update_fee()
        self.__update_value()

    def get_trade_info(self):
        return list(self.__dict__.values())


class TradeAnalysis:
    def __init__(self, seed_money:float=1_000):
        self.closed_trade :Dict[List[List[Any]]]={}
        self.open_trade :Dict[List[List[Any]]] = {} 

        #wallet 정보
        self.base_asset_value: float = seed_money  #기초자산
        self.asset_value: float = seed_money # 평가자(Total자산)
        self.trading_value:float = 0 #거래중인 자산
        self.available_balance: float = seed_money   #예수금
        self.profit_and_loss: float = 0 #손익금
        self.profit_loss_ratio: float = 0#손익률
        
    def add_closed_trade(self, trade_order_data:list):
        symbol = trade_order_data[0]
        if not symbol in self.closed_trade.keys():
            self.closed_trade[symbol] = [trade_order_data[1:]]
            # return self.closed_trade
        else:
            self.closed_trade[symbol].append(trade_order_data[1:])
        del self.open_trade[symbol]
        self.update_wallet()
        return self.closed_trade
    
    def update_open_trade(self, trade_order_data:list):
        symbol = trade_order_data[0]
        self.open_trade[symbol] = trade_order_data[1:]
        self.update_wallet()
        return self.open_trade

    def update_wallet(self):
        # 1차로 과거 거래내역을 wallet에 반영한다. open_trade가 없을 수 있으므로 모든 속성값을 업데이트한다.

        # 초기화가 필요한가?
        all_trades_data = []
        open_trades_data = []
        
        if self.closed_trade:
            for _, trades_close in self.closed_trade.items():
                all_trades_data.extend(trades_close)

        if self.open_trade:
            for _, trades_open in self.open_trade.items():
                all_trades_data.append(trades_open)
                open_trades_data.append(trades_open)
        
        if not self.closed_trade and not self.open_trade:
            self.trading_value = 0
            self.available_balance = self.asset_value
            return        
        
        # print(trade_data_list)
        
        all_trades_data_array = np.array(all_trades_data)
        open_trades_data_array = np.array(open_trades_data)
        # 매몰 비용 합계 (매수 수수료, 매도 수수료)
        
        if all_trades_data_array.ndim == 1:
            # 2차원 배열로 변환 (행 기준)
            all_trades_data_array = all_trades_data_array.reshape(1, -1)
            
        if open_trades_data_array.ndim == 1:
            # 2차원 배열로 변환 (행 기준)
            open_trades_data_array = open_trades_data_array.reshape(1, -1)
            
        
        #DEBUG
        print(open_trades_data_array)
        
        sunk_cost = np.sum(all_trades_data_array[:, [12, 13]])
        
        # print(trade_data_list)
        # print(sunk_cost)
        
        # 매수 비용 합계
        entry_cost = np.sum(all_trades_data_array[:, 7])
        # 비용 합계
        total_cost = sunk_cost + entry_cost
        # 회수 금액 합계(매도)
        recovered_value = np.sum(all_trades_data_array[:, 8])
        
        # 현재 거래중인 자산 초기화
        if open_trades_data_array.size > 0:
            self.trading_value = np.sum(open_trades_data_array[:,8])
        else:
            self.trading_value = 0
        # 예수금 반영(거래중인 자산이 없으므로)
        self.available_balance = self.base_asset_value - total_cost
        # 평가자산에서 회수 금액 추가
        self.asset_value = self.available_balance + self.trading_value
        # 손익평가 가치 반영(평가자산 - seed_money)
        self.profit_and_loss = np.sum(all_trades_data_array[:, 8])
        # 손익평가 가치 비율 환산(단위 : %)
        self.profit_loss_ratio = round((self.profit_and_loss / self.asset_value) * 100, 3)
        
            
class TradeOrderManager:
    def __init__(self, seed_money:float=1_000):
        self.active_orders: List[TradeOrder] = []
        self.symbol_map = {}
        self.trade_symbol: List = []
        
        # self.closed_trade = []
        # self.open_trade = []
        self.trade_analysis = TradeAnalysis(seed_money=seed_money)

    def add_order(self, **kwargs):
        
        # 거래중인 항목은 추가 거래 없음.
        if kwargs.get('symbol') in self.symbol_map.keys():
            return
        
        """TradeOrder 생성 및 추가"""
        valid_keys = {field.name for field in fields(TradeOrder)}  # TradeOrder 필드 이름 가져오기
        filtered_kwargs = {k: v for k, v in kwargs.items() if k in valid_keys}  # 유효한 키워드만 필터링
        order = TradeOrder(**filtered_kwargs)
        
        # print(order)
        
        self.active_orders.append(order)
        self.__update()  # symbol_map 업데이트
        
        order_signal = order.get_trade_info()
                
        self.trade_analysis.update_open_trade(order_signal)
        return order

    def remove_order(self, symbol):
        idx = self.symbol_map.get(symbol)
        order_signal = self.active_orders[idx].get_trade_info()
        self.trade_analysis.add_closed_trade(order_signal)
        del self.active_orders[idx]
        # del self.open_trade[symbol]
        self.__update()

    def update_data(self, symbol:str, current_price: [Union[float, int]], current_timestamp: int):
        symbol_idx = self.symbol_map.get(symbol)
        if symbol_idx is None:
            return
        self.active_orders[symbol_idx].update_data(current_price=current_price, current_timestamp=current_timestamp)
        trade_order_data = self.active_orders[symbol_idx].get_trade_info()
        self.trade_analysis.update_open_trade(trade_order_data)

    def get_order(self, symbol):
        idx = self.symbol_map.get(symbol)
        return self.active_orders[idx]

    def __update(self):
        """symbol_map 업데이트"""
        self.symbol_map = {order.symbol: idx for idx, order in enumerate(self.active_orders)}  # 심볼과 인덱스 매핑
        self.trade_symbol = list(self.symbol_map.keys())


In [2]:
obj = TradeOrderManager()
obj.add_order(symbol='ADAUSDT', start_timestamp=123, entry_price=0.2, position=1, quantity=1000, leverage=5)
# obj.update_data(symbol='ADAUSDT', current_price=2, current_timestamp=123)
obj.add_order(symbol='SOLUSDT', start_timestamp=123, entry_price=0.12, position=1, quantity=1000, leverage=5)
# obj.update_data(symbol='SOLUSDT', current_price=2, current_timestamp=123)
obj.add_order(symbol='XRPUSDT', start_timestamp=123, entry_price=0.501, position=1, quantity=1000, leverage=5)
# obj.update_data(symbol='XRPUSDT', current_price=2, current_timestamp=123)
obj.add_order(symbol='BTCUSDT', start_timestamp=123, entry_price=0.125, position=1, quantity=1000, leverage=5)
# obj.update_data(symbol='BTCUSDT', current_price=2, current_timestamp=123)
obj.remove_order(symbol='ADAUSDT')
obj.remove_order(symbol='SOLUSDT')
obj.remove_order(symbol='XRPUSDT')
obj.remove_order(symbol='BTCUSDT')

[[ 1.23e+02  2.00e-01  1.00e+00  1.00e+03  5.00e+00  5.00e-02  1.23e+02
   4.00e+01  4.00e+01 -1.00e+00  2.00e-01  2.01e-01  5.00e-01  5.00e-01]]
[[ 1.230e+02  2.000e-01  1.000e+00  1.000e+03  5.000e+00  5.000e-02
   1.230e+02  4.000e+01  4.000e+01 -1.000e+00  2.000e-01  2.010e-01
   5.000e-01  5.000e-01]
 [ 1.230e+02  1.200e-01  1.000e+00  1.000e+03  5.000e+00  5.000e-02
   1.230e+02  2.400e+01  2.400e+01 -6.000e-01  1.200e-01  1.206e-01
   3.000e-01  3.000e-01]]
[[ 1.23000e+02  2.00000e-01  1.00000e+00  1.00000e+03  5.00000e+00
   5.00000e-02  1.23000e+02  4.00000e+01  4.00000e+01 -1.00000e+00
   2.00000e-01  2.01000e-01  5.00000e-01  5.00000e-01]
 [ 1.23000e+02  1.20000e-01  1.00000e+00  1.00000e+03  5.00000e+00
   5.00000e-02  1.23000e+02  2.40000e+01  2.40000e+01 -6.00000e-01
   1.20000e-01  1.20600e-01  3.00000e-01  3.00000e-01]
 [ 1.23000e+02  5.01000e-01  1.00000e+00  1.00000e+03  5.00000e+00
   5.00000e-02  1.23000e+02  1.00200e+02  1.00200e+02 -2.50500e+00
   5.01000e-01  5.0

In [3]:
# obj.remove_order(symbol='BTCUSDT')
obj.trade_analysis.__dict__

{'closed_trade': {'ADAUSDT': [[123,
    0.2,
    1,
    1000,
    5,
    0.05,
    123,
    40.0,
    40.0,
    -1.0,
    0.2,
    0.201,
    0.5,
    0.5]],
  'SOLUSDT': [[123,
    0.12,
    1,
    1000,
    5,
    0.05,
    123,
    24.0,
    24.0,
    -0.6000000000000001,
    0.12,
    0.1206,
    0.30000000000000004,
    0.30000000000000004]],
  'XRPUSDT': [[123,
    0.501,
    1,
    1000,
    5,
    0.05,
    123,
    100.2,
    100.2,
    -2.505,
    0.501,
    0.503505,
    1.2525,
    1.2525]],
  'BTCUSDT': [[123,
    0.125,
    1,
    1000,
    5,
    0.05,
    123,
    25.0,
    25.0,
    -0.625,
    0.125,
    0.125625,
    0.3125,
    0.3125]]},
 'open_trade': {},
 'base_asset_value': 1000,
 'asset_value': 806.07,
 'trading_value': 0,
 'available_balance': 806.07,
 'profit_and_loss': -193.92999999999995,
 'profit_loss_ratio': -24.059}

In [ ]:
from dataclasses import dataclass, fields
from typing import List, Optional, Union, Any, Dict
import numpy as np
import copy

@dataclass
class TradeOrder:
    symbol: str
    start_timestamp: int  # 시작 시간
    entry_price: Union[float, int]  # 진입 가격
    position: int  # 포지션 (1: Long, -1: Short)
    quantity: Union[float, int]  # 수량
    leverage: int  # 레버리지
    fee_rate: float = 0.05  # 수수료율
    end_time: Optional[int] = None  # 종료 시간
    initial_value: Optional[float] = None  # 초기 가치
    current_value: Optional[float] = None  # 현재 가치
    profit_loss: Optional[float] = None  # 손익 금액
    current_price: Optional[Union[float, int]] = None  # 현재 가격
    break_even_price: Optional[float] = None  # 손익분기점 가격
    entry_fee: Optional[Union[float, int]] = None  # 진입 수수료
    exit_fee: Optional[Union[float, int]] = None  # 종료 수수료

    def __post_init__(self):
        if self.current_price is None:
            self.current_price = self.entry_price
        if self.leverage <= 0:
            raise ValueError(f"레버리지는 최소 1 이상이어야 합니다. 현재 값: {self.leverage}")
        if self.end_time is None:
            self.end_time = self.start_timestamp
        self.__update_fees()
        self.__update_values()

    def __update_fees(self):
        adjusted_fee_rate = self.fee_rate / 100
        self.entry_fee = self.entry_price * adjusted_fee_rate * self.quantity * self.leverage
        self.exit_fee = self.current_price * adjusted_fee_rate * self.quantity * self.leverage
        total_fees = self.entry_fee + self.exit_fee
        self.break_even_price = self.entry_price + (total_fees / self.quantity)

    def __update_values(self):
        self.initial_value = (self.entry_price * self.quantity) / self.leverage
        self.current_value = (self.current_price * self.quantity) / self.leverage
        total_fees = self.entry_fee + self.exit_fee
        self.profit_loss = self.current_value - self.initial_value - total_fees

    def update_trade_data(self, current_price: Union[float, int], current_time: int):
        self.current_price = current_price
        self.end_time = current_time
        self.__update_fees()
        self.__update_values()

    def to_list(self):
        return list(self.__dict__.values())


class TradeAnalysis:
    def __init__(self, initial_balance: float = 1_000):
        self.closed_positions: Dict[str, List[List[Any]]] = {}
        self.open_positions: Dict[str, List[List[Any]]] = {}

        self.initial_balance: float = initial_balance  # 초기 자산
        self.total_balance: float = initial_balance  # 총 평가 자산
        self.active_value: float = 0  # 거래 중 자산 가치
        self.cash_balance: float = initial_balance  # 사용 가능한 예수금
        self.profit_loss: float = 0  # 손익 금액
        self.profit_loss_ratio: float = 0  # 손익률

    def add_closed_position(self, trade_order_data: list):
        symbol = trade_order_data[0]
        if symbol not in self.closed_positions:
            self.closed_positions[symbol] = [trade_order_data[1:]]
        else:
            self.closed_positions[symbol].append(trade_order_data[1:])
        del self.open_positions[symbol]
        self.update_wallet()
        return self.closed_positions

    def update_open_position(self, trade_order_data: list):
        symbol = trade_order_data[0]
        self.open_positions[symbol] = trade_order_data[1:]
        self.update_wallet()
        return self.open_positions

    def update_wallet(self):
        trade_data_list = []

        if self.closed_positions:
            for _, trades_close in self.closed_positions.items():
                trade_data_list.extend(trades_close)

        if self.open_positions:
            for _, trades_open in self.open_positions.items():
                trade_data_list.append(trades_open)

        if not self.closed_positions and not self.open_positions:
            self.active_value = 0
            self.cash_balance = self.total_balance
            return

        trade_data_array = np.array(trade_data_list)
        if trade_data_array.ndim == 1:
            trade_data_array = trade_data_array.reshape(1, -1)

        sunk_cost = np.sum(trade_data_array[:, [12, 13]])
        entry_cost = np.sum(trade_data_array[:, 7])
        total_cost = sunk_cost + entry_cost
        recovered_value = np.sum(trade_data_array[:, 8])

        self.active_value = entry_cost
        self.cash_balance = self.initial_balance - total_cost
        self.total_balance = self.cash_balance + self.active_value
        self.profit_loss = np.sum(trade_data_array[:, 9])
        self.profit_loss_ratio = round((self.profit_loss / self.initial_balance) * 100, 3)
        
        
        if self.cash_balance < 0:
            raise ValueError(f'주문 오류 : 진입금액이 예수금을 초과함.')
        
        # 계좌 가치가 0 미만이면 청산신호 발생 및 프로그램 중단.
        if self.total_balance < 0:
            raise ValueError(f'청산발생 : 계좌잔액 없음.')


class TradeOrderManager:
    def __init__(self, initial_balance: float = 1_000):
        self.active_orders: List[TradeOrder] = []
        self.order_index_map = {}
        self.active_symbols: List[str] = []
        self.trade_analysis = TradeAnalysis(initial_balance=initial_balance)

    def __refresh_order_map(self):
        self.order_index_map = {order.symbol: idx for idx, order in enumerate(self.active_orders)}
        self.active_symbols = list(self.order_index_map.keys())

    def add_order(self, **kwargs):
        if kwargs.get('symbol') in self.order_index_map:
            return

        valid_keys = {field.name for field in fields(TradeOrder)}
        filtered_kwargs = {k: v for k, v in kwargs.items() if k in valid_keys}
        order = TradeOrder(**filtered_kwargs)

        self.active_orders.append(order)
        self.__refresh_order_map()

        order_signal = order.to_list()
        self.trade_analysis.update_open_position(order_signal)
        return order

    def remove_order(self, symbol: str):
        idx = self.order_index_map.get(symbol)
        order_signal = self.active_orders[idx].to_list()
        self.trade_analysis.add_closed_position(order_signal)
        del self.active_orders[idx]
        self.__refresh_order_map()

    def update_order_data(self, symbol: str, current_price: Union[float, int], current_time: int):
        idx = self.order_index_map.get(symbol)
        if idx is None:
            return
        self.active_orders[idx].update_trade_data(current_price=current_price, current_time=current_time)
        trade_order_data = self.active_orders[idx].to_list()
        self.trade_analysis.update_open_position(trade_order_data)

    def get_order(self, symbol: str):
        idx = self.order_index_map.get(symbol)
        return self.active_orders[idx]

In [ ]:
obj = TradeOrderManager()
obj.add_order(symbol='ADAUSDT', start_timestamp=123, entry_price=0.5, position=1, quantity=1000, leverage=5)
# obj.update_data(symbol='ADAUSDT', current_price=2, current_timestamp=123)
obj.add_order(symbol='SOLUSDT', start_timestamp=123, entry_price=0.5102, position=1, quantity=1000, leverage=5)
# obj.update_data(symbol='SOLUSDT', current_price=2, current_timestamp=123)
obj.add_order(symbol='XRPUSDT', start_timestamp=123, entry_price=0.1052, position=1, quantity=1000, leverage=5)
# obj.update_data(symbol='XRPUSDT', current_price=2, current_timestamp=123)
obj.add_order(symbol='BTCUSDT', start_timestamp=123, entry_price=0.235, position=1, quantity=1000, leverage=5)
# obj.update_data(symbol='BTCUSDT', current_price=2, current_timestamp=123)
obj.remove_order(symbol='ADAUSDT')
obj.remove_order(symbol='SOLUSDT')
obj.remove_order(symbol='XRPUSDT')
obj.remove_order(symbol='BTCUSDT')

In [ ]:
obj.add_order(symbol='ADAUSDT', start_timestamp=123, entry_price=0.1, position=1, quantity=1000, leverage=5)
obj.update_order_data(symbol='ADAUSDT', current_price=1, current_time=124)

In [ ]:
obj.trade_analysis.__dict__

In [ ]:
obj.remove_order(symbol='ADAUSDT')

In [ ]:
obj.trade_analysis.__dict__